In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from mlp import Layer, MultilayerPerceptron, SquaredError, Relu, Linear
from sklearn.metrics import r2_score

# -------------------- Loading MPG Dataset -------------------- #
class MPGDataLoader:
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path

    def load_data(self):
        columns = ["mpg", "cylinders", "displacement", "horsepower", "weight",
                   "acceleration", "model year", "origin", "car name"]
        df = pd.read_csv(self.dataset_path, sep='\s+', names=columns, na_values="?")
        df.dropna(inplace=True)
        df.drop(columns=["car name"], inplace=True, errors='ignore')
        df["horsepower"] = pd.to_numeric(df["horsepower"], errors='coerce')
        df.dropna(inplace=True)

        y = df["mpg"].values.reshape(-1, 1)
        X = df.drop(columns=["mpg"]).values
        return X, y

# Load dataset
dataset_path = 'mpg-data/auto-mpg.data'
dataloader = MPGDataLoader(dataset_path)
X, y = dataloader.load_data()

# -------------------- Data Preprocessing -------------------- #
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True)

scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_x.fit_transform(X_train)
X_val = scaler_x.transform(X_val)
X_test = scaler_x.transform(X_test)

y_train = scaler_y.fit_transform(y_train)
y_val = scaler_y.transform(y_val)
y_test = scaler_y.transform(y_test)

# -------------------- Hyperparameter Tuning: Network Depth -------------------- #
input_dim = X_train.shape[1]

depth_configurations = [
    [Layer(input_dim, 64, Relu(), dropout_rate=0.0), Layer(64, 1, Linear())],  # 1 hidden layer
    [Layer(input_dim, 64, Relu(), dropout_rate=0.0), Layer(64, 32, Relu(), dropout_rate=0.0), Layer(32, 1, Linear())],  # 2 hidden layers
    [Layer(input_dim, 64, Relu(), dropout_rate=0.0), Layer(64, 32, Relu(), dropout_rate=0.0), Layer(32, 16, Relu(), dropout_rate=0.0), Layer(16, 1, Linear())],  # 3 hidden layers
    [Layer(input_dim, 64, Relu(), dropout_rate=0.0), Layer(64, 32, Relu(), dropout_rate=0.0), Layer(32, 16, Relu(), dropout_rate=0.0), Layer(16, 8, Relu(), dropout_rate=0.0), Layer(8, 1, Linear())]  # 4 hidden layers
]

results = {}

for idx, layers in enumerate(depth_configurations):
    depth = len(layers) - 1  # Number of hidden layers
    print(f"\nTraining model with {depth} hidden layers...")

    mlp = MultilayerPerceptron(layers)
    train_losses, val_losses = mlp.train(
        X_train, y_train, X_val, y_val,
        SquaredError(), learning_rate=0.001, batch_size=32, epochs=150, optimizer='vanilla', momentum=0.7
    )

    y_pred = mlp.forward(X_test, training=False)

    # Ensure y_pred is reshaped correctly before inverse transformation
    y_pred = y_pred.reshape(-1, 1)
    y_pred = scaler_y.inverse_transform(y_pred)

    # Also inverse-transform y_test for accurate comparison
    y_test_original = scaler_y.inverse_transform(y_test)

    # Compute R² Score
    r2 = r2_score(y_test_original, y_pred)

    # Store results
    results[depth] = round(r2 * 100, 2)  # Convert to percentage for better readability
    print(f"R² Score for {depth} hidden layers: {results[depth]}%")

# Identify Best Depth Configuration
best_depth = max(results, key=results.get)
best_r2 = results[best_depth]

# -------------------- Summary of Results -------------------- #
print("\n===== Summary of Runs =====")
for depth, r2 in results.items():
    print(f"R² value for {depth} hidden layers: {r2}%")

print(f"\nBest Depth Configuration: {best_depth} hidden layers with R² = {best_r2}%")



Training model with 1 hidden layers...
Epoch 1/150 - Train Loss: 0.8010, Val Loss: 0.5863
Epoch 2/150 - Train Loss: 0.5817, Val Loss: 0.4324
Epoch 3/150 - Train Loss: 0.4654, Val Loss: 0.3447
Epoch 4/150 - Train Loss: 0.3526, Val Loss: 0.2909
Epoch 5/150 - Train Loss: 0.3050, Val Loss: 0.2595
Epoch 6/150 - Train Loss: 0.2633, Val Loss: 0.2405
Epoch 7/150 - Train Loss: 0.2465, Val Loss: 0.2273
Epoch 8/150 - Train Loss: 0.2321, Val Loss: 0.2175
Epoch 9/150 - Train Loss: 0.2223, Val Loss: 0.2094
Epoch 10/150 - Train Loss: 0.2154, Val Loss: 0.2025
Epoch 11/150 - Train Loss: 0.2049, Val Loss: 0.1963
Epoch 12/150 - Train Loss: 0.2000, Val Loss: 0.1906
Epoch 13/150 - Train Loss: 0.1978, Val Loss: 0.1854
Epoch 14/150 - Train Loss: 0.1900, Val Loss: 0.1801
Epoch 15/150 - Train Loss: 0.1862, Val Loss: 0.1748
Epoch 16/150 - Train Loss: 0.1811, Val Loss: 0.1706
Epoch 17/150 - Train Loss: 0.1764, Val Loss: 0.1662
Epoch 18/150 - Train Loss: 0.1761, Val Loss: 0.1623
Epoch 19/150 - Train Loss: 0.1671